<a href="https://colab.research.google.com/github/soccy/Study/blob/main/hongong_03_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 특성 공학과 규제: 여러 특성을 사용사용한 다중 회귀에 대해 배우고, 사이킷런의 여러 도구를 사용해 보자
# 복잡한 모델의 과대적합을 막기 위한 릿지(Ridge)와 라쏘(Lasso) 회귀를 배워보자

 (추가 설명 - 인공지능을 위한 수학)
 주어진 데이터데이터에 너무 정확히 들여맞아 지나치게 복잡하게 표현된 상태를 "과적합, 과학습" 이라고 하기도 한다. 반면 약간의 노이즈는 허용하면서도 전체적인 데이터의 특성을 잘 반영하고 있는 식을 "일반화 능력(generalization ablility)"이 있다라고 말한다. 당연히 인공지능 모델에서는 일반화 능력이 있는 모델을 만들어야하고, 그러기 위해서는 과학습이 발생하지 않도록 잘 조절해줘야 한다.

  선형회귀에서는 과학습을 피하는 방법으로 정규화(regularization)라는 것이 있다. 정규화는 모델이 복잡해질 수록 일종의 페널티를 적용하는 방법으로 과학습을 억제한다. 좀 더 구체적으로 말하자면 모델이 지나치게 복잡해지지 않도록 모델식 자체에 모델의 계수(가중치)가 작아지게 만드는 항을 추가하는 방법을 사용한다. 정규화를 할때는 보통 L1정규화와 L2정규화를 사용한다. (page234 공식참조)

  L1 정규화에서는 정규화 항이 최소화함수 D를 최소화할수있는 가중치W 를 찾다보면 WW의 노름(Norm)이 너무 커지지 않도록 억제가 되며, 그 말은 곧 모델이 너무 복잡해지지 않도록 억제한다는 말이기도 하다. 이렇게 정규화된 선형회귀를 Lasso 회귀라고 한다.

  L2 정규화에서는 L1L1정규화와는 달리 절댓값을 사용하지 않기때문에 상대적으로 미분이 쉬워진다. 이렇게 정규화된 선형회귀를 Ridge 회귀라고한다.

  L1 정규화와 L2 정규화는 기존의 모델식에 조합해서 쓸 수 있고, 심지어 이 둘을 조합할 수도 있는데 그렇게 만들어진 회귀 모델을 Elastic Net 이라고 한다. 참고로 정규화에 사용된 남다는 상수인데 남다값을 크게 잡으면 전체를 최소화하기 위해 가중치 함수의 노름이 작아진다. 즉, 정규화를 더 강화시키는 효과가 있다. 결국 남다를 조정하면 정규화의 강약을 조절할수 있다. scikit-learn 에서는 특별히 지정하지 않는한 기본적으로 남다=1.0 으로 계산한다.

# 3-2 에서 배운, 선형회귀(Linear regression) case 의 경우, train set 보다 test set 점수가 높았던 것이 조금 찜찜하다. 이 문제를 해결하해결하려면 제곱보다 더 고차항을 넣어야할 것 같은데(왜냐하면, 선형회귀하면서 제곱항을 하나 더 넣어서 점수가 좋아졌었음) 얼만큼 더 고차항을 넣어야할 지 모르고 수동으로 이렇게 고차항을 넣기도 힘들다.

# 받은 데이터가 길이만 있었던게 아니고, 높이와 두께도 줬었다 ==> 빙고 ==> 받은 데이터 모두를 사용하자. 선형회귀는 특성이 많을수록 효과가 좋다. 높이와 두께를 다항 회귀와 함께 적용해 보자 ==> 수고롭게 직접하지 말고, 사이킷런의 polynomialFeatures 클래스를 사용하자.

# 다중회귀 (multiple regression) - 여러개의 특성을 사용한 선형선형회귀
# 3-2 에서 1 개의 특성을 사용했을 때, 선형회귀 모델이 학습한 것은 ==> 직선 (물론, 제곱 활용해서 2 차 함수형식으로 만들기도 했었다)
# 2 개의 특성을 사용하면 ==> 선형회귀 ==> 평면을 학습한다 ==> 3 차원 공간을 형성하게 됨 ==>"타깃 = a * 특성 1 + b * 특성 2 + 절편"
# 3 개의 특성을 사용하면 ==> 안타깝지만 우리는 3 차원 공간 이상을 그리거나 상상할 수는 없다. 분명한 것은 선형 회귀를 단순한 직선이나 평면으로 생각하여 성능이 무조건 낮다고 오해해서는 안된다. 특성이 많은 고차원에서는 선형 회귀가 매우 복잡한 모델을 표현할수 있다.

# 이 예제에서는 농어의 길이, 높이, 두께 모두를 사용한다. ==> 또한 이전 절에서처럼 3 개의 특성을 각각 제곱하여 추가한다. ==> 거기다가 각 특성을 서로 곱해서 또 다른 특성을 만들겠다.
# 즉, "농어길이 * 농어높이"를 새로운 특성으로 만든다. 이렇게 기존의 특성을 사용해 새로운 특성을 뽑아내는 작업을 "특성공학 (feature engineering)" 이라 부른다.

# 먼저, 데이터를 불러들이자
# pandas ==> 유명한 데이터 분석 libarary ==> dataframe: pandas 의 핵심 데이터 구조 <== numpy 배열과 비슷하게 다차원 배열을 다룰 수 있지만, 훨씬 많은 기능을 제공함. 또한 data frame 은 numpy 배열로 쉽게 바꿀수 도 있다.
# pandas 를 이용해서, 농어데이터를 인터넷에서 다운받아 dataframe 에 저장하자 ==> 그 다음 numpy 배열로 변환하여 선형 회귀 모델을 train 하자. 먼저,dataframe 을 만들기 위해 많이 사용하는 파일은 csv 파일이다.


In [3]:
import pandas as pd

df = pd.read_csv('https://bit.ly/perch_csv_data')

perch_full = df.to_numpy()

print(perch_full)
print('-----------------------')
print(perch_full.shape)

[[ 8.4   2.11  1.41]
 [13.7   3.53  2.  ]
 [15.    3.82  2.43]
 [16.2   4.59  2.63]
 [17.4   4.59  2.94]
 [18.    5.22  3.32]
 [18.7   5.2   3.12]
 [19.    5.64  3.05]
 [19.6   5.14  3.04]
 [20.    5.08  2.77]
 [21.    5.69  3.56]
 [21.    5.92  3.31]
 [21.    5.69  3.67]
 [21.3   6.38  3.53]
 [22.    6.11  3.41]
 [22.    5.64  3.52]
 [22.    6.11  3.52]
 [22.    5.88  3.52]
 [22.    5.52  4.  ]
 [22.5   5.86  3.62]
 [22.5   6.79  3.62]
 [22.7   5.95  3.63]
 [23.    5.22  3.63]
 [23.5   6.28  3.72]
 [24.    7.29  3.72]
 [24.    6.38  3.82]
 [24.6   6.73  4.17]
 [25.    6.44  3.68]
 [25.6   6.56  4.24]
 [26.5   7.17  4.14]
 [27.3   8.32  5.14]
 [27.5   7.17  4.34]
 [27.5   7.05  4.34]
 [27.5   7.28  4.57]
 [28.    7.82  4.2 ]
 [28.7   7.59  4.64]
 [30.    7.62  4.77]
 [32.8  10.03  6.02]
 [34.5  10.26  6.39]
 [35.   11.49  7.8 ]
 [36.5  10.88  6.86]
 [36.   10.61  6.74]
 [37.   10.84  6.26]
 [37.   10.57  6.37]
 [39.   11.14  7.49]
 [39.   11.14  6.  ]
 [39.   12.43  7.35]
 [40.   11.93

In [5]:
import os
print(os.getcwd())

/content


In [9]:
# Target 데이터는 이전과 동일한 방식으로 준비
import numpy as np

perch_weight = np.array([5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0,
       115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
       150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
       218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
       556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
       850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
       1000.0])

print(perch_weight)
print('-----------------------')
print(perch_weight.shape)

[   5.9   32.    40.    51.5   70.   100.    78.    80.    85.    85.
  110.   115.   125.   130.   120.   120.   130.   135.   110.   130.
  150.   145.   150.   170.   225.   145.   188.   180.   197.   218.
  300.   260.   265.   250.   250.   300.   320.   514.   556.   840.
  685.   700.   700.   690.   900.   650.   820.   850.   900.  1015.
  820.  1100.  1000.  1100.  1000.  1000. ]
-----------------------
(56,)


In [12]:
# perch_full(length, height, width), perch_weight 를 train set, test set 으로 나눈다.

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, random_state=42)

print(train_input, train_input.shape)
print('-----------------------')
print(test_input, test_input.shape)
print('-----------------------')
print(train_target, train_target.shape)
print('-----------------------')
print(test_target, test_target.shape)

[[19.6   5.14  3.04]
 [22.    5.88  3.52]
 [18.7   5.2   3.12]
 [17.4   4.59  2.94]
 [36.   10.61  6.74]
 [25.    6.44  3.68]
 [40.   11.93  7.11]
 [39.   12.43  7.35]
 [43.   11.93  7.28]
 [22.    5.64  3.52]
 [20.    5.08  2.77]
 [22.    6.11  3.52]
 [24.    7.29  3.72]
 [27.5   7.17  4.34]
 [43.   12.51  7.42]
 [40.   11.73  7.22]
 [24.    6.38  3.82]
 [21.    5.92  3.31]
 [27.5   7.05  4.34]
 [40.   12.38  7.46]
 [32.8  10.03  6.02]
 [26.5   7.17  4.14]
 [36.5  10.88  6.86]
 [13.7   3.53  2.  ]
 [22.7   5.95  3.63]
 [15.    3.82  2.43]
 [37.   10.57  6.37]
 [35.   11.49  7.8 ]
 [28.7   7.59  4.64]
 [23.5   6.28  3.72]
 [39.   11.14  6.  ]
 [21.    5.69  3.56]
 [23.    5.22  3.63]
 [22.    5.52  4.  ]
 [44.   12.49  7.6 ]
 [22.5   6.79  3.62]
 [19.    5.64  3.05]
 [37.   10.84  6.26]
 [22.    6.11  3.41]
 [25.6   6.56  4.24]
 [42.   12.8   6.87]
 [34.5  10.26  6.39]] (42, 3)
-----------------------
[[ 8.4   2.11  1.41]
 [18.    5.22  3.32]
 [27.5   7.28  4.57]
 [21.3   6.38  3.53]
 

In [20]:
##### 공부 용도

# sklearn 은 특성을 만들거나 전처리하기 위한 다양한 클래스를 제공한다. 사이킷런에서는 이런 클래스를 변환기 (transfomer)라고 부른다.
# sklearn 모델 class: fit(), score(), predict()
# sklearn 변환기 class: fit(), transform() 메서드를 제공함.
# 우리가 이번에 사용한 변환기(transformer) 는 PolynomialFeature 클래스임

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()

poly.fit([[2, 3]])                # 훈련을 해야
print(poly.transform([[2, 3]]))     # 변환도 한다. 사이킷런의 일관된 api 때문에 두 단계로 나뉘어져 있다. 두 메서드를 하나로 붙인 fit_transform 메서드도 있다.

# fit() 메서드는 새롭게 만들 특성 조합을 찾고(눈여겨봐라, transformer(변환기)는 입력 데이터를 변환하는데, target 데이터는 불필요하다.)
# transform() 메서드는 실제로 데이터를 변환한다.
# 상기 예제는 22개의 특성을 가진 샘플이 66개의 특성을 가진 샘플로 바뀐다.
# PolynomialFeatures 클래스는 기본적으로 특성을 제곱한 항을 추가하고, 특성기리 서로 곱한 항을 추가한다. 2 와 3 을 각각 제곱한 4, 9 가 추가되었고
# 2 와 3 을 곱한 6 이 추가되었다. 1 은 왜 추가되었을까 ==> 무게 = a* 길이 + b * 높이 + c * 두께 + d * 1

# 사실, 선형 방정식의 절편을 항상 값이 1 인 특성과 곱해지는 계수라고 볼 수 있다. 이렇게 놓고보면 특성은 (길이, 높이, 두께, 1) 이 된다.
# 하지만, 사이킷런의 선형 모델은 자동으로 절편이 추가되므로, 굳이 이렇게 특성을 만들 필요가 없다. include_bias = False 로 지정하여 다시 특성 변환을 하자.
poly = PolynomialFeatures(include_bias=False)
poly.fit([[2, 3]])
print(poly.transform([[2, 3]])) # 절편 항(bias) 이 제거되고, 특성의 제곱과 특성끼리 곱한 항만 결국 추가되었다.

# include_bias=False 로 지정하지 않아도, sklearn 모델은 자동으로 특성에 추가된 절편을 무시한다. 하지만 여기서는 혼돈을 피라기 위해 명시적으로 지정한다.


[[1. 2. 3. 4. 6. 9.]]
[[2. 3. 4. 6. 9.]]


In [26]:
# train input 에 적용하자.
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)                          # (42, 3)인 train_input 을 훈련시키고(33개의 특성이다: length, height, wwidth)

train_poly = poly.transform(train_input)      # 변환한다. (==> tain_poly 에 저장)
print(train_input.shape)
print(train_poly.shape)
print('--------------------')

# 9 개의 특성이 어떻게 만들어졌는지 확인해보자
poly.get_feature_names_out()

(42, 3)
(42, 9)
--------------------


array(['x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2',
       'x2^2'], dtype=object)